In [1]:
%config InlineBackend.figure_format = 'retina' 

In [2]:

from entanglement import Entanglement_Capability
from expressibility import Expressibility

from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import TwoLocal


In [3]:
two = TwoLocal(3, 'ry', 'cx', 'linear', reps=2, insert_barriers=True)

In [4]:
print(two)

     ┌──────────┐ ░            ░ ┌──────────┐ ░            ░ ┌──────────┐
q_0: ┤ Ry(θ[0]) ├─░───■────────░─┤ Ry(θ[3]) ├─░───■────────░─┤ Ry(θ[6]) ├
     ├──────────┤ ░ ┌─┴─┐      ░ ├──────────┤ ░ ┌─┴─┐      ░ ├──────────┤
q_1: ┤ Ry(θ[1]) ├─░─┤ X ├──■───░─┤ Ry(θ[4]) ├─░─┤ X ├──■───░─┤ Ry(θ[7]) ├
     ├──────────┤ ░ └───┘┌─┴─┐ ░ ├──────────┤ ░ └───┘┌─┴─┐ ░ ├──────────┤
q_2: ┤ Ry(θ[2]) ├─░──────┤ X ├─░─┤ Ry(θ[5]) ├─░──────┤ X ├─░─┤ Ry(θ[8]) ├
     └──────────┘ ░      └───┘ ░ └──────────┘ ░      └───┘ ░ └──────────┘


In [5]:
descriptor1 = Entanglement_Capability(two)
descriptor2 = Expressibility(two)

In [6]:
descriptor1.samples

10

In [7]:
print("The entanglement capability of the given circuit is: ", descriptor1.get_result())
print("The expressibility of the given circuit is: ", descriptor2.get_result())

The entanglement capability of the given circuit is:  0.4212171441574767
The expressibility of the given circuit is:  0.08849674487551594


In [8]:
Num = 4
qc = QuantumCircuit(Num)
x = ParameterVector(r'$\theta$', length=8)

[qc.h(i) for i in range(Num)]   
[qc.ry(x[int(2*i)], i) for i in range(Num)]
[qc.rz(x[int(2*i+1)], i) for i in range(Num)]

qc.cx(0, range(1, Num))
qc.draw("mpl")

<Figure size 621.941x367.889 with 1 Axes>

In [9]:
qc.parameters

ParameterView([ParameterVectorElement($\theta$[0]), ParameterVectorElement($\theta$[1]), ParameterVectorElement($\theta$[2]), ParameterVectorElement($\theta$[3]), ParameterVectorElement($\theta$[4]), ParameterVectorElement($\theta$[5]), ParameterVectorElement($\theta$[6]), ParameterVectorElement($\theta$[7])])

In [11]:
descriptor1 = Entanglement_Capability(qc,samples=10)
descriptor2 = Expressibility(qc,samples=10)

In [12]:
print("The entanglement capability of the given circuit is: ", descriptor1.get_result())
print("The expressibility of the given circuit is: ", descriptor2.get_result())

The entanglement capability of the given circuit is:  0.3320884004357065
The expressibility of the given circuit is:  0.40958042025849106


In [13]:
class Interface:
    
    def __init__(self,circ=None,samples:int=1000)->None:
        """
        Interface which initialise the user input.
        ------------------------------------------
        circ: qiskit.QuantumCircuit, a user defined quantum circuit, must contain parametrised quantum gates
        samples: int, number of samples we want to draw for the parameters.
        """
        self._circ = circ 
        self._num_qubits = circ.num_qubits
        self._samples = samples
        
        
    

In [14]:
inter = Interface(two)
print(inter._num_qubits)

3


In [114]:

class Entanglement_Capability(Interface):
    
    def __init__(self,circ=None,samples:int=1000,method_ec:str='mw'):
        super().__init__(circ,samples)
        self._method_ec = method_ec

    @staticmethod
    def scott_helper(state, perms):
        
        dems = np.linalg.matrix_power(
            [partial_trace(state, list(qb)).data for qb in perms], 2
        )
        trace = np.trace(dems, axis1=1, axis2=2)
        return np.sum(trace).real

    def meyer_wallach_measure(self,states, num_qubits):
        r"""Returns the meyer-wallach entanglement measure for the given circuit.
        .. math::
            Q = \frac{2}{|\vec{\theta}|}\sum_{\theta_{i}\in \vec{\theta}}
            \Bigg(1-\frac{1}{n}\sum_{k=1}^{n}Tr(\rho_{k}^{2}(\theta_{i}))\Bigg)
        """
        num_qubits = self._num_qubits
        permutations = list(itertools.combinations(range(num_qubits), num_qubits - 1))
        ns = 2 * sum(
            [
                1 - 1 / num_qubits * self.scott_helper(state, permutations)
                for state in states
            ]
        )
        
        return ns.real
 
    def scott_measure(self,states, num_qubits):
        r"""Returns the scott entanglement measure for the given circuit.
        .. math::
            Q_{m} = \frac{2^{m}}{(2^{m}-1) |\vec{\theta}|}\sum_{\theta_i \in \vec{\theta}}\
            \bigg(1 - \frac{m! (n-m)!)}{n!}\sum_{|S|=m} \text{Tr} (\rho_{S}^2 (\theta_i)) \bigg)\
            \quad m= 1, \ldots, \lfloor n/2 \rfloor
        """
        num_qubits = self._num_qubits
        m = range(1, num_qubits // 2 + 1)
        permutations = [
            list(itertools.combinations(range(num_qubits), num_qubits - idx))
            for idx in m
        ]
        combinations = [1 / comb(num_qubits, idx) for idx in m]
        contributions = [2 ** idx / (2 ** idx - 1) for idx in m]
        ns = []

        for ind, perm in enumerate(permutations):
            ns.append(
                contributions[ind]
                * sum(
                    [
                        1 - combinations[ind] * self.scott_helper(state, perm)
                        for state in states
                    ]
                )
            )

        return np.array(ns)
    
  
    

In [115]:
ec = Entanglement_Capability(circ=two)
print(ec._method_ec)
print(ec._circ)

mw
     ┌──────────┐ ░            ░ ┌──────────┐ ░            ░ ┌──────────┐
q_0: ┤ Ry(θ[0]) ├─░───■────────░─┤ Ry(θ[3]) ├─░───■────────░─┤ Ry(θ[6]) ├
     ├──────────┤ ░ ┌─┴─┐      ░ ├──────────┤ ░ ┌─┴─┐      ░ ├──────────┤
q_1: ┤ Ry(θ[1]) ├─░─┤ X ├──■───░─┤ Ry(θ[4]) ├─░─┤ X ├──■───░─┤ Ry(θ[7]) ├
     ├──────────┤ ░ └───┘┌─┴─┐ ░ ├──────────┤ ░ └───┘┌─┴─┐ ░ ├──────────┤
q_2: ┤ Ry(θ[2]) ├─░──────┤ X ├─░─┤ Ry(θ[5]) ├─░──────┤ X ├─░─┤ Ry(θ[8]) ├
     └──────────┘ ░      └───┘ ░ └──────────┘ ░      └───┘ ░ └──────────┘


In [116]:
import numpy as np

from scipy.spatial import distance

In [183]:
class Expressibility(Interface):
    
    def __init__(self,circ=None,samples:int=1000,method_ex:str='kl'):
        super().__init__(circ,samples)
        self._method_ex = method_ex
        print("Entering expressibility init ...")
        print("method ex: ",method_ex)


    @staticmethod
    def kl_divergence(prob_a: np.ndarray, prob_b: np.ndarray) -> float:
        """Returns KL divergence between two probabilities"""
        prob_a[prob_a == 0] = 1e-10
        kl_div = np.sum(np.where(prob_a != 0, prob_a * np.log(prob_a / prob_b), 0))
        return typing.cast(float, kl_div)
    
    @staticmethod
    def js_distance(prob_a: np.ndarray, prob_b: np.ndarray,base: float=2.)-> float:
        return distance.jessenshannon(prob_a,prob_b,base)
    
    def prob_haar(self) -> np.ndarray:
        """Returns probability density function of fidelities for Haar Random States"""
        fidelity = np.linspace(0, 1, self._samples)
        num_qubits = self._num_qubits
        
        return (2 ** num_qubits - 1) * (1 - fidelity + 1e-8) ** (2 ** num_qubits - 2)

    

In [184]:
dsc = Expressibility(two)
dsc._method_ex

Entering expressibility init ...
method ex:  kl


'kl'

In [319]:
import typing
import qiskit
import itertools
from qiskit.quantum_info import partial_trace
from qiskit.quantum_info import state_fidelity
from multiprocessing import Process, Queue, Value, Array

class Multiprocessing(Expressibility,Entanglement_Capability):
    
    def __init__(self,descriptor:str='ex',
                 num_core:int=-1,circ=None,samples:int=10,
                 method_ex:str='kl',method_ec:str='mw'):
        """
        Create a multiprocessing pool to handle job with multi cores.
        -------------------------------------------------------------
        descriptor: string, specify the descriptor(s) of interest, can 
        be "ex", "ec" or "both".
        num_core: number of cores used in the job.
        
        """
     
        #Interface.__init__(circ=circ,samples=samples,
        #                          mwthod_ex=method_ex,method_ec=method_ec)
        if descriptor == "ex":
            print("entering expressibility...")
            print("method_ex: ",method_ex)
            Expressibility.__init__(self,circ,samples,method_ex)
        elif descriptor == "ec":
            Entanglement_Capability.__init__(self,circ,samples,method_ec)
        elif descriptor == "both":
            Expressibility.__init__(self,circ,samples,method_ex)
            Entanglement_Capability.__init__(self,circ,samples,method_ec)
            
            
    def simulate(self,theta):
        circuit =  self._circ.assign_parameters(theta)
        circuit.snapshot("final", snapshot_type="statevector")
        result = qiskit.execute(
            circuit, qiskit.Aer.get_backend("aer_simulator_statevector")
        ).result()
        result_data = result.data(0)["snapshots"]["statevector"]["final"][0]

        return result_data
    
    def get_params(self) -> typing.Tuple[typing.List, typing.List]:
        """Generate parameters for the calculation of expressibility
        :returns theta (np.array): first list of parameters for the parameterized quantum circuit
        :returns phi (np.array): second list of parameters for the parameterized quantum circuit
        """
        
        theta = [
            {p: 2 * np.random.random() * np.pi for p in self._circ.parameters}
            for _ in range(self._samples)
        ]
        phi = [
            {p: 2 * np.random.random() * np.pi for p in self._circ.parameters}
            for _ in range(self._samples)
        ]
        
        params = theta,phi
        
        return params
    
    
    def get_circuits(self):
        """
        Function that needs multiprocessing.
        
        """
        
        thetas, phis = self.get_params()
        theta_circuits = [
                self.simulate(theta)
                for theta in thetas
        ]
            
        phi_circuits = [
            self.simulate(phi)
            for phi in phis
        ]
        
        return np.vstack(theta_circuits), np.vstack(phi_circuits)
    
    def prob_pqc(self, shots: int = 1024) -> np.ndarray:
        """Return probability density function of fidelities for PQC
        :param shots: number of shots for circuit execution
        :returns fidelities (np.array): np.array of fidelities
        """
        theta_circuits, phi_circuits = self.get_circuits()
        fidelity = np.array(
            [
                state_fidelity(rho_a, rho_b)
                for rho_a, rho_b in itertools.product(theta_circuits, phi_circuits)
            ]
        )
        
        return np.array(fidelity)
    
    
    def get_entanglement(self):
        num_qubits = self._num_qubits
        theta_circuits, phi_circuits = self.get_circuits()
        method = self._method_ec
        if method=='mw':
            
            circ_entanglement_capability = self.meyer_wallach_measure(
                        theta_circuits + phi_circuits, num_qubits
                    ) / (2 * self._samples)
            
        elif method=='scott':
            
            circ_entanglement_capability = self.scott_measure(
                        theta_circuits + phi_circuits, num_qubits
                    ) / (2 * self._samples)
            
        else:
            raise ValueError("Invalid measure provided, choose from 'mw' or 'scott'")
            
        return circ_entanglement_capability
    
    def get_expressibility(self, shots: int = 1024):
        samples = self._samples
        measure = self._method_ex
        haar = self.prob_haar()
        haar_prob: np.ndarray = haar / float(haar.sum())

        if len(self._circ.parameters) > 0:
            fidelity = self.prob_pqc(shots)
        else:
            fidelity = np.ones(self._samples ** 2)

        bin_edges: np.ndarray
        pqc_hist, bin_edges = np.histogram(
            fidelity, samples, range=(0, 1), density=True
        )
        pqc_prob: np.ndarray = pqc_hist / float(pqc_hist.sum())

        if measure == "kl":
            pqc_expressibility = self.kl_divergence(pqc_prob, haar_prob)
        elif measure == "js":
            pqc_expressibility = self.jensenshannon(pqc_prob, haar_prob)
        else:
            raise ValueError("Invalid measure provided, choose from 'kl' or 'js'")
        #self.plot_data = [haar_prob, pqc_prob, bin_edges]
        #self.expr = pqc_expressibility

        return pqc_expressibility
    
    def job(self):
        pass
    
    def target(self,):
        
        while not q.empty():
        ind = q.get()
        P, M = run_one(ind,h=.7,ratio=ratio, cratio=cratio, alpha=alpha)
        if len(P) != 1 and len(M) != 1:
            arr1[count1.value*6:(count1.value+int(len(P)/6))*6] = P
            arr2[count2.value*3:(count2.value+int(len(M)/3))*3] = M
            count1.value += int(len(P)/6)
            count2.value += int(len(M)/3)
        
    
    
            
        

In [320]:
job = Multiprocessing(circ=two,samples=4,descriptor="both")

Entering expressibility init ...
method ex:  kl


In [321]:
job.get_expressibility()

0.19956419691548655

In [322]:
out = job.get_circuits()

In [323]:
t,p = out
t.shape

(4, 8)

In [293]:
tt = np.vstack(t)
t.vstack()

AttributeError: 'list' object has no attribute 'vstack'

In [222]:
from multiprocessing import Process, Queue, Value, Array

a = np.arange(10000)
num_proc = 4
def target(a):
    return np.random.choice(a)

def qinit(q, index):
    for i in index:
        q.put(i)

process_list = []
index = range(100)
q = Queue(len(index))
qinit(q, index)
for i in range(num_proc):
    p = Process(target=target,args=(a,))
    p.start()
    process_list.append(p)
for i in process_list:
    p.join() 

[<Process(Process-5, stopped)>,
 <Process(Process-6, stopped)>,
 <Process(Process-7, stopped)>,
 <Process(Process-8, stopped)>]